In [ ]:
#============================================= Libraries ==============================================================
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
#============================================= Data Path ==============================================================
path_feat = ""
path_feat2 = ""
path_out = ""

In [ ]:
#============================================= Data Import ============================================================
df1 = pd.read_csv(path_feat)
df2 = pd.read_csv(path_feat2)
df = pd.concat([df1,df2])
df.reset_index(drop=True, inplace=True)
df.date = df.date.apply(lambda x: pd.to_timedate(x[0:7], format='%Y-%m'))
df = df.groupby(['stock','date'], as_index=False).first()

In [ ]:
#============================================= Functions ==============================================================
def make_reg_variable(df, names, nd):
    for name in names:
        df['{}_trend_{}'.format(name,nd)] = df['{}'.format(name)].rolling(nd).corr(df['Unnamed: 0'])
        df['{}_trend_{}'.format(name,nd*2)] = df['{}'.format(name)].rolling(nd*2).corr(df['Unnamed: 0'])
        df['{}_trend_{}'.format(name,nd*5)] = df['{}'.format(name)].rolling(nd*5).corr(df['Unnamed: 0'])
    return df

def make_change_variable(df, names):
    for name in names:
        df['{}_change'.format(name)] = df['{}'.format(name)]/df['{}'.format(name)].shift(1) - 1
    return df

def make_12_trailing(df, names):
    for name in names:
        df['{}_12m_trail'.format(name)] = df['{}'.format(name)].rolling(12).mean()
    return df

def resample_3mo(df):
    df.set_index('date', inplace=True)
    y = df.resample('3m').last()
    return y

def resample_1mo(df):
    df.set_index('date', inplace=True)
    y = df.resample('BM').pad()
    y = y.fillna(method='ffill')
    return y

def last_day_of_month(date):
    if date.month == 12:
        return date.replace(day=31)
    return date.replace(month=date.month+1, day=1) - timedelta(days=1)

In [ ]:
#============================================= Data Cleaning ==========================================================
df.date = df.date.apply(lambda x: last_day_of_month(x))
df = df.groupby('stock', as_index=False).apply(resample_3mo)
df['EBITDARD'] = df.EBITDA + df.IS_RD_EXPEND

col_reg = ['SPS','OCF','GPOA','RETURN_COM_EQY']
col_rate = ['DSI', 'DSR', 'TURNOVER', 'BS_TOT_ASSET', 'AQI', 'DEPI', 'GMI', 'DEBT']
col_trail = ['EBIT', 'CAPEX_ABSOLUTE_VALUE', 'EXTERN_EQY_FNC', 'SALES_REV_TURN', 'NET_INCOME',
            'CF_CASH_FROM_OPER', 'CF_FREE_CASH_FLOW', 'EBITDA', 'EBITDARD', 'NET_INCOME']
col_vol = ['ACC']

df = df.groupby('stock', as_index=False).apply(make_12_trailing, col_trail)

In [ ]:
#============================================= Define Variables =======================================================
df['DE'] = df.SHORT_AND_LONG_TERM_DEBT/df.TOT_COMMON_EQY
df['DEBITDA'] = df.SHORT_AND_LONG_TERM_DEBT/df.EBITDA
df['LTDE'] = df.BS_LT_BORROW/df.TOT_COMMON_EQY
df['LTDEBITDA'] = df.BS_LT_BORROW/df.EBITDA
df['NDE'] = df.NET_DEBT/df.TOT_COMMON_EQY
df['NDEBITDA'] = df.NET_DEBT/df.EBITDA
df['EBITCOVER'] = df.EBIT_12m_trail/df.IS_INT_EXPENSE
df['SPS'] = df.SALES_REV_TURN/df.BS_SH_OUT
df['OCF'] = df.CF_CASH_FROM_OPER/df.BS_SH_OUT
df['GPOA'] = df.GROSS_PROFIT/df.BS_TOT_ASSET
df['ACCTAF'] = (df.NET_INCOME - df.CF_CASH_FROM_OPER)/df.BS_TOT_ASSET
df['DSI'] = df.BS_INVENTORIES/df.SALES_REV_TURN
df['DSR'] = df.BS_ACCT_NOTE_RCV/df.SALES_REV_TURN
df['TURNOVER'] = df.BS_TOT_ASSET/df.SALES_REV_TURN
df['AQI'] = df.BS_CUR_ASSET/df.SALES_REV_TURN
df['DEPI'] = df.IS_DEPR_EXP/df.FIXED_ASSETS_AND_LT_INVESTMENTS
df['ACC'] = df.BS_ACCRUAL/df.BS_TOT_ASSET
df['GMI'] = df.GROSS_PROFIT/df.SALES_REV_TURN
df['DEBT'] = df.SHORT_AND_LONG_TERM_DET/df.BS_TOT_ASSET
df['CAPEXPPE'] = df.CAPEX_ABSOLUTE_VALUE_12m_trail/df.FIXED_ASSETS_AND_LT_INVESTMENTS
df['EXTFA'] = df.EXTERN_EQY_FNC_12m_trail/df.BS_TOT_ASSET
df['PSALES'] = df.SALES_REV_TURN_12m_trail/df.CUR_MKT_CAP
df['PE'] = df.NET_INCOME_12m_trail/df.CUR_MKT_CAP
df['POCF'] = df.CF_CASH_FROM_OPER_12m_trail/df.CUR_MKT_CAP
df['PFCF'] = df.CF_FREE_CASH_FLOW_12m_trail/df.CUR_MKT_CAP
df['EVSALES'] = df.SALES_REV_TURN_12m_trail/df.CURR_ENTP_VAL
df['EVEBIT'] = df.EBIT_12m_trail/df.CURR_ENTP_VAL
df['EVFCF'] = df.CF_FREE_CASH_FLOW_12m_trail/df.CURR_ENTP_VAL
df['EVOCF'] = df.CF_CASH_FROM_OPER_12m_trail/df.CURR_ENTP_VAL
df['EVEBITDA'] = df.EBITDA_12m_trail/df.CURR_ENTP_VAL
df['EVEBITDARD'] = df.EBITDARD_12m_trail/df.CURR_ENTP_VAL
df['PB'] = df.TOT_COMMON_EQY/df.CUR_MKT_CAP
df['PTB'] = (df.TOT_COMMON_EQY - df.BS_DISCLOSED_INTANGIBLES)/df.CUR_MKT_CAP
df['ROE'] = df.NET_INCOME_12m_trail/df.TOT_COMMON_EQY
df['ROA'] = df.NET_INCOME_12m_trail/df.BS_TOT_ASSET
df['OPRDA'] = df.EBITDARD_12m_trail/df.BS_TOT_ASSET
df['ROIC'] = df.NET_INCOME_12m_trail/df.TOTAL_INVESTED_CAPITAL
df['OPRDIC'] = df.EBITDARD_12m_trail/df.TOTAL_INVESTED_CAPITAL
df['FCFROIC'] = df.CF_FREE_CASH_FLOW_12m_trail/df.TOTAL_INVESTED_CAPITAL

In [ ]:
df = df.groupby('stock', as_index=False).apply(make_reg_variable, col_reg, 4)
df = df.groupby('stock', as_index=False).apply(make_change_variable, col_rate)

In [ ]:
df.to_csv(path_out)